In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate
!pip install rouge
!pip install rouge_score
!pip install bert_score
!pip install gradio
!pip install typing_extensions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [ ]:
from typing_extensions import Doc

In [ ]:

import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

import gradio as gr
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

In [ ]:

df_train=pd.read_excel("/content/aiml-qa-train.xlsx")
df_test = pd.read_excel("/content/aiml-qa-test.xlsx")

In [ ]:
df_train.head(10)

,question,answer
0,What is the difference between concatenation v...,Concatenation combines two tensors by adding t...
1,What is the difference between concatenation v...,Concatenation is often used to combine differe...
2,Why are derivatives substracted from weights?,The derivative of the loss function at a point...
3,Why are derivatives substracted from weights?,The intuition behind adjusting the weights by ...
4,Describe a process/pipeline for generating rep...,Select a pretrained model suitable for the tas...
5,Describe a process/pipeline for generating rep...,Load a pretrained model and remove its final c...
6,Explain the differencr between Feedback & back...,"The term ""feedback"" is not standard in context..."
7,Explain the differencr between Feedback & back...,Feedback is a more general concept that includ...
8,what technique is used for kernel transformat...,Kernel transformation in SVMs uses a non-linea...
9,what technique is used for kernel transformat...,Standardizing the data to have zero mean and u...


In [ ]:
df_test.head(10)

,question,answer1,answer2
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...
5,Adding layer to a machine learning model can l...,Increasing the number of layers may lead to ov...,The architecture of the model depends on the d...
6,is comparision of model predictions important?,Ensuring the model's right prediction in real ...,"Yes, comparing model outputs against ground tr..."
7,What is the way to invert the association rules?,"Yes, by inverting association rules, we can id...","To invert association rules, identify items wi..."
8,What is the good number of filters we can have?,The number of output channels in a CNN layer c...,The number of filters in a CNN layer depends o...
9,How are Part-of-Speech (POS) tags learned in n...,POS tags are learned using supervised or unsup...,POS tags can be learned through supervised lea...


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"  #"gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [ ]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

# default to left padding
tokenizer.padding_side = "left"

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df_train.iterrows():
    question = row['question']
    answer = row['answer']
    question = " ".join(question.split()[:300])
    out = "<Question>\n" + str(question) + " \nAnswer\n " + str(answer) +" \n<ANSWER_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/gpt_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,2.251200


TrainOutput(global_step=987, training_loss=1.963561269649981, metrics={'train_runtime': 146.7433, 'train_samples_per_second': 26.904, 'train_steps_per_second': 6.726, 'total_flos': 159108475392000.0, 'train_loss': 1.963561269649981, 'epoch': 3.0})

In [ ]:
model_output_path = "/content/model"

In [ ]:
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

In [ ]:
def generate_answer(question):
    input_ids = tokenizer.encode(question, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    start_index = decoded_output.find("Answer")
    end_index = decoded_output.find("<ANSWER_ENDED>")

    if end_index != -1:
        # Extract the text between "Answer" and "<ANSWER_ENDED>"
        answer_text = decoded_output[start_index + len("Answer"):end_index].strip()
        return answer_text
    else:
        # If "<ANSWER_ENDED>" is not found, return the text following "Answer"
        answer_text = decoded_output[start_index + len("Answer"):].strip()
        return answer_text

    #return tokenizer.decode(output[0], skip_special_tokens=True)
    #return tokenizer.decode(output, skip_special_tokens=True)

In [ ]:
model_output_path='/content/drive/MyDrive/QAModel_D1'

In [ ]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_output_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/QAModel_D1'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#model[0].to("cuda")
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
question="What is deep learning?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Deep learning is a subset of artificial intelligence that focuses on extracting features from data and then applying machine learning algorithms to learn and interpret the data.


In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Generating answers")
df_test['generated_answers'] = df_test['question'].progress_apply(lambda question: generate_answer(question))

Generating answers: 100%|██████████| 120/120 [02:02<00:00,  1.02s/it]


In [ ]:
df_test.head(5)

,question,answer1,answer2,generated_answers
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te...","Yes, converting 2D images to 1D can be done by..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali...","Yes, we can utilize an autoencoder to perform ..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...,NLP is a powerful tool for both research and c...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str...","Yes, the problem of understanding the hidden d..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM scaling, but..."


In [ ]:
import re

def extract_answer(text):
    pattern = r'Answer(.*?)<ANSWER_ENDED>'
    result = re.search(pattern, text, re.DOTALL)

    if result:
        return result.group(1).strip()
    else:
        return text

In [ ]:
df_test['generated_answers'] = df_test['generated_answers'].apply(extract_answer)

In [ ]:
df_test.head(5)

,question,answer1,answer2,generated_answers
0,How we can effectively convert 2D images to 1D?,Converting images to 1D data may not be effect...,"To effectively convert 2D images to 1D, use te...","Yes, converting 2D images to 1D can be done by..."
1,Can we utilize an autoencoder to perform dimen...,"Yes, autoencoders can be applied to numerical ...","Yes, autoencoders can be used for dimensionali...","Yes, we can utilize an autoencoder to perform ..."
2,What is NLP's current biggest challenge that i...,The main challenges of NLP is finding and coll...,NLP models struggle with tasks that require re...,NLP is a powerful tool for both research and c...
3,Which problems cannot be solved by Neural netw...,While neural networks have shown great success...,"Neural networks are powerful, but they may str...","Yes, the problem of understanding the hidden d..."
4,Is scaling necessary for SVM?,"Yes, scaling the input data is generally recom...",Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM scaling, but..."


In [ ]:
df_test.to_csv('model_Qtest_data_output.csv', index=False)

In [ ]:
df_test=pd.read_csv('/content/model_Qtest_data_output.csv')

In [ ]:
df_test.columns

Index(['question', 'answer1', 'answer2', 'generated_answers'], dtype='object')

In [ ]:
from evaluate import load
from collections import defaultdict
from tqdm import tqdm
from bert_score import score

def metric(data):
    Question,Answer,Generated_Answer  = ["question","answer1","generated_answers"]
    rouge = load("rouge")
    meteor = load("meteor")
    score_bert = defaultdict(int)
    bertscore = load("bertscore")

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_ans = rouge.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["rouge_1_ans"] += rouge_ans['rouge1']
        score_bert["rouge_2_ans"] += rouge_ans['rouge2']
        score_bert["rouge_L_ans"] += rouge_ans['rougeL']

        meteor_ans = meteor.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["meteor_ans"] += meteor_ans["meteor"]

        # BERT score
        results = bertscore.compute(predictions=[x[Generated_Answer]], references=[x[Answer]], lang="en")

        score_bert["bert_score_f1"] += results["f1"][0]

    print("The evalution scores are as follows:")

    for name,val in score_bert.items():
        print(f"{name}: {val/len(data)}")




In [ ]:
metric(df_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
  0%|          | 0/120 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 120/120 [00:42<00:00,  2.85it/s]

The evalution scores are as follows:
rouge_1_ans: 0.29671961169483624
rouge_2_ans: 0.11129691253708231
rouge_L_ans: 0.2459883221345918
meteor_ans: 0.2486321058753435
bert_score_f1: 0.8785953218738238
